In [ ]:
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cmocean as cmo
import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy as np
import copy
np.seterr(all='ignore');

In [ ]:
run = 'n011'

fnames = sorted(glob.glob(f'../data/ecefiles/{run}/{run}*_T.nc'))

year0 = np.array([int(fname[-27:-23]) for fname in fnames])
year1 = np.array([int(fname[-18:-14]) for fname in fnames])
nyears = sum(year1+1-year0)

basin = np.arange(0,5)
bnames = ['EAIS','WEDD','AMUN','ROSS','APEN']
bcol = ['tab:green','tab:blue','tab:red','tab:orange','tab:purple']

In [ ]:
#Get time-independent variables
ds = xr.open_dataset('../data/ecefiles/areas.nc')
area = ds['O1t0.srf'].values;
ds.close()

ds = xr.open_dataset(fnames[0])
lat = ds['nav_lat'].values
lon = ds['nav_lon'].values
ds.close()

In [ ]:
#Get weighted mask for basin averages
lons = lon.copy()
lats = lat.copy()

mask = np.repeat(np.zeros(lons.shape)[np.newaxis,:,:],len(basin),axis=0)
aweight = np.zeros(mask.shape)

for b,bb in enumerate(basin):
    mm = np.zeros(lons.shape)
    if b==0:
        #EAIS
        mm[((lons<173) & (lons>-10)) & (lats<-65) & (lats>-76)] = 1
    if b==1:
        #WEDD
        mm[(lons>-65) & (lons<-10) & (lats<-72)] = 1
    if b==2:
        #AMUN
        mm[(lons>-150) & (lons<-80) & (lats<-70)] = 1
    if b==3:
        #ROSS
        mm[((lons>150) | (lons<-150)) & (lats<-76)] = 1
    if b==4:
        #APEN
        mm[(lons>-66) & (lons<-56) & (lats>-70) & (lats<-65)] = 1
        mm[(lons>-80) & (lons<-65) & (lats>-75) & (lats<-70)] = 1
        
    mask[b,:,:] = mm
    aweight[b,:,:] = mask[b,:,:]*area

In [ ]:
T = np.zeros((12*nyears,len(basin)))
S = np.zeros((12*nyears,len(basin)))
Z = np.zeros((12*nyears,len(basin)))

#Calculate basin-average time series
ttime = []

m = 0
for f,fname in enumerate(fnames):
    ds = xr.open_dataset(fname)
    time = ds['time_centered'].values
    tos = ds['tos'].values
    sos = ds['sos'].values
    zos = ds['zos'].values
    ds.close()
    
    for t,tt in enumerate(time):
        ttime.append(tt)
        for b,bb in enumerate(bnames):
            T[m*12+t,b] = np.nansum(tos[t,:,:]*aweight[b,:,:])/np.nansum(aweight[b,:,:])
            S[m*12+t,b] = np.nansum(sos[t,:,:]*aweight[b,:,:])/np.nansum(aweight[b,:,:])
            Z[m*12+t,b] = np.nansum(zos[t,:,:]*aweight[b,:,:])/np.nansum(aweight[b,:,:])
    m += 1
ttime = np.array(ttime).flatten()

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(131)
for b in basin:
    ax.plot(ttime,T[:,b],c=bcol[b],label=bnames[b])
ax.legend()
ax = fig.add_subplot(132)
for b in basin:
    ax.plot(ttime,S[:,b],c=bcol[b])
ax = fig.add_subplot(133)
for b in basin:
    ax.plot(ttime,Z[:,b],c=bcol[b])
    print(f'trend {bnames[b]}:',1000*np.polyfit(np.arange(0,Z.shape[0]), Z[:,b], 2))
plt.show()

In [ ]:
#Plot spatial difference between start and end

Nyr = 15



for vname in ['zos','tos','sos']:

    if vname == 'zos':
        vmin = -2
        vmax = -1
        vdif = .2
        cmap = copy.copy(mpl.cm.get_cmap("cmo.amp"));
    elif vname == 'sos':
        vmin = 33
        vmax = 34.5
        vdif = 1
        cmap = copy.copy(mpl.cm.get_cmap("cmo.haline"));
    elif vname == 'tos':
        vmin = -2
        vmax = 4
        vdif = 2
        cmap = copy.copy(mpl.cm.get_cmap("cmo.thermal"));

    proj = ccrs.SouthPolarStereo()
    cmap2 = copy.copy(mpl.cm.get_cmap("cmo.delta"));

    fig = plt.figure(figsize=(21,5))


    ds = xr.open_dataset(fnames[0])
    var1 = np.nanmean(ds[vname].values[:Nyr*12,:,:],axis=0)
    ds.close()

    ax = fig.add_subplot(131,projection=proj)
    ax.set_extent([-280, 80, -80, -60], crs=ccrs.PlateCarree())
    im = ax.scatter(lon,lat,50,c=var1,cmap=cmap,transform=ccrs.PlateCarree(),lw=0,vmin=vmin,vmax=vmax)
    ax.coastlines(lw=.1)
    ax.set_title(f'First {Nyr} years')

    gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=False)
    gl.xlocator = mticker.FixedLocator(np.arange(-180,210,30))
    gl.xformatter = LONGITUDE_FORMATTER
    plt.colorbar(im)

    ds = xr.open_dataset(fnames[-1])
    var2 = np.nanmean(ds[vname].values[-Nyr*12:,:,:],axis=0)
    ds.close()

    ax = fig.add_subplot(132,projection=proj)
    ax.set_extent([-280, 80, -80, -60], crs=ccrs.PlateCarree())
    im = ax.scatter(lon,lat,50,c=var2,cmap=cmap,transform=ccrs.PlateCarree(),lw=0,vmin=vmin,vmax=vmax)
    ax.coastlines(lw=.1)
    ax.set_title(f'Last {Nyr} years')

    gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=False)
    gl.xlocator = mticker.FixedLocator(np.arange(-180,210,30))
    gl.xformatter = LONGITUDE_FORMATTER
    plt.colorbar(im)

    var3 = var2-var1

    ax = fig.add_subplot(133,projection=proj)
    ax.set_extent([-280, 80, -80, -60], crs=ccrs.PlateCarree())
    im = ax.scatter(lon,lat,50,c=var3,cmap=cmap2,transform=ccrs.PlateCarree(),lw=0,vmin=-vdif,vmax=vdif)
    ax.coastlines(lw=.1)
    ax.set_title(f'difference last - first')

    gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=False)
    gl.xlocator = mticker.FixedLocator(np.arange(-180,210,30))
    gl.xformatter = LONGITUDE_FORMATTER


    plt.colorbar(im)
    plt.show()